In [1]:
import pandas as pd
import re
from operator import itemgetter
import ast
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network
import networkx.algorithms.community as nx_com
from sklearn.preprocessing import MinMaxScaler

In [2]:
class Sna:
    def sna(data):    
        # List untuk menyimpan tweet yang mengandung tag '@' dan usernamenya
        tagged_tweets = []
        # print(tagged_tweets)

        # Ekstraksi tweet yang mengandung tag '@' dan usernamenya
        for row in data:
            full_text = row['full_text']
            username = row['username']

            # Gunakan ekspresi reguler untuk mencari tag '@' dan usernamenya
            mentions = re.findall(r'@(\w+)', full_text)
            # print(mentions)
            # Jika ada tag '@', tambahkan ke list tagged_tweets
            if mentions:
                for mention in mentions:
                    # print(mention)
                    tagged_tweets.append({'source': username, 'target': mention, 'tweet': full_text})

        # Buat DataFrame dari list tagged_tweets
        df = pd.DataFrame(tagged_tweets)
        print(df)

        # List untuk menyimpan data dengan atribut weight
        weighted_data = []

        # Atribut weight dasar
        base_weight = 1

        # Ekstraksi data dan hitung weight
        for index, row in df.iterrows():
            source = row['source']
            target = row['target']

            # Cari apakah data dengan source dan target yang sama sudah ada di weighted_data
            existing_data_index = None
            for i, data in enumerate(weighted_data):
                if data['source'] == source and data['target'] == target:
                    existing_data_index = i
                    break

            if existing_data_index is not None:
                # Jika data sudah ada, tambahkan 1 pada atribut weight
                weighted_data[existing_data_index]['weight'] += 1
            else:
                # Jika data belum ada, tambahkan data baru dengan weight dasar
                weighted_data.append({'source': source, 'target': target, 'weight': base_weight})

        # Buat DataFrame dari list weighted_data
        weighted_df = pd.DataFrame(weighted_data)

        # Buat graf dari DataFrame
        G = nx.from_pandas_edgelist(weighted_df, 'source', 'target', ['weight'])

        # Hitung betweenness centrality
        bc_values = nx.betweenness_centrality(G)

        # Hitung eigenvector centrality
        ev_values = nx.eigenvector_centrality(G)

        # Buat DataFrame dari hasil betweenness centrality
        bc_df = pd.DataFrame(list(bc_values.items()), columns=['account_name', 'BC'])

        # Urutkan DataFrame berdasarkan kolom 'BC' secara descending
        bc_df = bc_df.sort_values(by='BC', ascending=False)

        # Buat DataFrame dari hasil eigenvector centrality
        ev_df = pd.DataFrame(list(ev_values.items()), columns=['account_name', 'EV'])

        # Urutkan DataFrame berdasarkan kolom 'EV' secara descending
        ev_df = ev_df.sort_values(by='EV', ascending=False)

        # Normalisasi menggunakan Min-Max Scaling untuk kolom 'EV'
        scaler_ev = MinMaxScaler(feature_range=(0, 1))
        ev_df['norm_EV'] = ev_df['EV'].rank() / len(ev_df)

        # Normalisasi menggunakan Min-Max Scaling untuk kolom 'BC'
        scaler_bc = MinMaxScaler()
        bc_df['norm_BC'] = bc_df['BC'].rank() / len(bc_df)

        # Gabungkan DataFrame bc_df dan ev_df berdasarkan kolom 'account_name'
        result_df = pd.merge(bc_df, ev_df, on='account_name', how='outer')

        # Susun kolom sesuai dengan urutan yang diinginkan
        result_df = result_df[['account_name', 'BC', 'EV', 'norm_BC', 'norm_EV']]

        # Buat atribut baru 'final_measure' yang berisikan rata-rata dari 'norm_BC' dan 'norm_EV'
        result_df['final_measure'] = (result_df['norm_BC'] + result_df['norm_EV']) / 2

        # Urutkan DataFrame berdasarkan kolom 'final_measure' secara descending
        result_df = result_df.sort_values(by='final_measure', ascending=False)

        # Ambil top 10 nilai 'final_measure'
        top_10_final_measure = result_df.head(10)

        # Tampilkan tabel dengan urutan nomor dan account_name
        top_10_final_measure_table = top_10_final_measure[['account_name']].reset_index(drop=True)
        top_10_final_measure_table.index += 1

        # Create a networkx graph
        G = nx.from_pandas_edgelist(df, 'source', 'target')

        # Ekstrak top 10 account names dari DataFrame
        top_account_names = top_10_final_measure['account_name'].tolist()

        # Define the scale factor for node sizes
        scale_factor_top = 30  # Adjust this value as needed
        scale_factor_default = 15  # Adjust this value as needed for default nodes

        # Define the color for the nodes
        node_color = "#EEB3B3"

        # Create a pyvis network object
        graph = Network(notebook=True)

        # Add nodes and edges to the pyvis graph with specific size and color
        for node in G.nodes:
             # Ambil tweet yang terkait dengan node ini
            tweets = df[df['source'] == node]['tweet'].tolist()
            # Gabungkan semua tweet ke dalam satu string HTML
            tweet_html = '<br>'.join(tweets)
    
            size = scale_factor_top if node in top_account_names else scale_factor_default
            graph.add_node(node, size=size, color=node_color, title=tweets)

        for edge in G.edges:
            print(edge)
            graph.add_edge(edge[0], edge[1])

        # Set default options
        options = {
            "nodes": {
                "shape": 'dot',
                "font": {
                    "size": 12
                }
            }
        }

        # Convert options to a JSON-formatted string
        options_str = str(options).replace("'", '"')


        # Show the graph with options
        graph.set_options(options_str)
        graph.show('graph.html')
        
        return top_10_final_measure_table

In [3]:
data = pd.read_csv('./dataset/dataset-testing/costrx-moisturizer-keyword-thread.csv', sep=',')
data

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1.800000e+18,Tue May 28 21:59:28 +0000 2024,0,@nanawourld yg iniii pls bagus bgt! klo ga sal...,1.800000e+18,https://pbs.twimg.com/media/GOsqhc4bcAYL2hv.jpg,nanawourld,in,֪ ֪ ֪,0,1,0,https://x.com/qfloryn/status/1795575645927534898,2.315959e+09,qfloryn
1,1.800000e+18,Mon May 27 23:40:08 +0000 2024,0,@ohmybeautybank kalo tekstur sih iyaa bisa tp ...,1.800000e+18,NaN,ohmybeautybank,in,NaN,0,0,0,https://x.com/dayooee/status/1795238588822728895,8.520000e+17,dayooee
2,1.790000e+18,Mon May 27 00:48:44 +0000 2024,0,Kayanya moist cosrx kemarin terlalu heavy deh ...,1.790000e+18,NaN,chrispypaw,in,stayville,0,0,0,https://x.com/chrispypaw/status/17948934651108...,1.190000e+18,chrispypaw
3,1.790000e+18,Sun May 26 11:46:58 +0000 2024,0,@tanyarlfes moist cosrx 200k. alias lumayan bg...,1.790000e+18,NaN,tanyarlfes,in,notknown,0,0,0,https://x.com/jalapinokio/status/1794696727351...,1.220000e+18,jalapinokio
4,1.790000e+18,Sun May 26 01:13:13 +0000 2024,0,skincare aku hari ini : hadalabo face wash amp...,1.790000e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/alyyucanread/status/179453723939...,1.700000e+18,alyyucanread
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,1.770000e+18,Mon Apr 01 03:46:48 +0000 2024,0,@ohmybeautybank PLS MOIST INI BGS BGT DI AKU!!...,1.770000e+18,NaN,ohmybeautybank,in,NaN,0,0,0,https://x.com/pearlintheseaaa/status/177464455...,1.380000e+18,pearlintheseaaa
415,1.770000e+18,Mon Apr 01 03:04:36 +0000 2024,0,@ohmybeautybank Di aku yg dry yg moist ini kur...,1.770000e+18,NaN,ohmybeautybank,in,Unknown,0,0,0,https://x.com/sun_flowwie/status/1774633936087...,1.520000e+18,sun_flowwie
416,1.770000e+18,Mon Apr 01 01:16:29 +0000 2024,0,@ohmybeautybank kulitku type combi tp lebih ke...,1.770000e+18,NaN,ohmybeautybank,in,NaN,0,0,0,https://x.com/mamiuwil/status/1774606727536943354,1.590000e+18,mamiuwil
417,1.770000e+18,Mon Apr 01 14:23:01 +0000 2024,0,@ohmybeautybank Buat gue yg kulitnya suka dehi...,1.770000e+18,NaN,ohmybeautybank,in,Bandung,0,0,0,https://x.com/andtydt/status/1774804666293055839,1.120849e+09,andtydt


In [4]:
dataForSna = []
for index, tweet in data.iterrows():
    dataForSna.append({
        "username": tweet['username'],
        "full_text": tweet['full_text']
    })


In [5]:
buzzer = Sna.sna(dataForSna)['account_name'].values.tolist()

              source          target  \
0            qfloryn      nanawourld   
1            dayooee  ohmybeautybank   
2        jalapinokio      tanyarlfes   
3       urstrawbewwy      beauthingy   
4       woo_everyone    unnesmenfess   
..               ...             ...   
356  pearlintheseaaa  ohmybeautybank   
357      sun_flowwie  ohmybeautybank   
358         mamiuwil  ohmybeautybank   
359          andtydt  ohmybeautybank   
360          szeanvm  ohmybeautybank   

                                                 tweet  
0    @nanawourld yg iniii pls bagus bgt! klo ga sal...  
1    @ohmybeautybank kalo tekstur sih iyaa bisa tp ...  
2    @tanyarlfes moist cosrx 200k. alias lumayan bg...  
3    @beauthingy St aku normal to oily pake moist c...  
4    @unnesmenfess moist cosrx snail mucin bkin glo...  
..                                                 ...  
356  @ohmybeautybank PLS MOIST INI BGS BGT DI AKU!!...  
357  @ohmybeautybank Di aku yg dry yg moist ini kur...  
358  @

In [6]:
buzzer

['ohmybeautybank',
 'beautales_',
 'cupidciu',
 'untukcariduit',
 'piiingss',
 'markepret',
 'schiaparelIIi',
 'awaa__wa',
 'Purplelight____',
 'ayanav_']

In [7]:
# # Membuat dictionary untuk menyimpan tweet berdasarkan nama pengguna
# tweets_by_user = {}

# # Mengelompokkan tweet berdasarkan nama pengguna
# for tweet_data in dataForSna:
#     username = tweet_data['username']
#     full_text = tweet_data['full_text']
    
#     if username in tweets_by_user:
#         tweets_by_user[username].append(full_text)
#     else:
#         tweets_by_user[username] = [full_text]

# # Menampilkan tweet yang berhubungan dengan setiap buzzer
# for buzzer_username in buzzer:
#     print(f"Tweet yang berhubungan dengan {buzzer_username}:")
#     if buzzer_username in tweets_by_user:
#         for tweet in tweets_by_user[buzzer_username]:
#             print(tweet)
#     else:
#         print("Tidak ada tweet yang berhubungan.")



Tweet yang berhubungan dengan idextratime:
Half-time: Arab Saudi 0-0 Korea Selatan  Lebih meyakinkan mainnya timnas Indonesia ga sih  #Blibli @bliblidotcom
Tweet yang berhubungan dengan bliblidotcom:
Tidak ada tweet yang berhubungan.
Tweet yang berhubungan dengan moechaaliem:
@ghozali0312 @indosupporter IS dulu pernah punya tim yg keren diantara semua kelompok umur timnas yakni u19 eranya Evan Dimas, smpe saat ini bhkn belum nemu lagi yg kyak gitu Mainnya pun bnyak bola pendek, squad pun dibentuk dri keliling nyari pemain, knpa skrng ngoyo ya? Apa mungkin bnyakan pemain titipan"
@NurHasa52632749 @erirendy5 @Indostransfer Tpi di era itu timnas kita msih menangan klo ketemu mereka, beda jauh ketika dilatih si engkong, hasil paling bagus ya seri, itupun di kandang kita mainnya
@cco_archive @gberrei @Indostransfer Ngapain nyerempet ama Oman, lhaa mereka aja mainnya pekok msak mau ngarep lolos 🤣 Posisi lagi nyari gol aja pemainnya pda lemes, mnding timnas kita lah yg lolos
@MafiaWasit Psti 

In [8]:
# # Daftar buzzer yang ingin Anda periksa tweet-nya

# # Iterasi untuk setiap buzzer
# for buz in buzzer:
#     print(f"Tweet yang berisi tag untuk akun {buz}:")
#     for tweet_data in dataForSna:
#         full_text = tweet_data['full_text']
#         mentions = re.findall(r'@(\w+)', full_text)
#         if buz in mentions or tweet_data['username'] == buz:
#             print(full_text)
#     print("\n")


Tweet yang berisi tag untuk akun idextratime:
@idextratime @bliblidotcom Ngeri bgt gaya mainnya haye , mimpi apa timnas punya diaspora kek gini 😭
@meta_parad44820 @idextratime Liat nanti wkwk. Diaspora era etho blm ada yg main dan liga nya msh kacau. Prestasi yg km banggakan itu cm timnas yg dilatih STY. yg bahkan cara mainnya udh berkembang dan bagus di AFF 2021 Singapore. Liga ga bikin etho kesorot cm timnas. Dan klo ngikutin timnas mah itu kerja STY
@Unterhose08 @Opariorio @idextratime @EmiratesFACup Enzo di timnas cemerlang mainnya karena ada Mac Allister, pun dengan si Caicedo mainnya bagus karena ada Mac Allister pas main di Brighton.  Nah skrng 2 orng itu tandeman di Chelsea. Mainnya masih on off, klo lg bagus ya bagus. Tp lebih sering off perform..angin2an blm konsisten
@yourmaolanaa @noershall @Iniozan9 @idextratime wkwkwk ngeliat ettorito kek mainnya timnas italia jaman 2000an  gol 1, langsung bertahan, ulur waktu😂🤣
@idextratime ini mainnya harus pakai timnas masing masing ya

In [30]:
# class Sna:
#     @staticmethod
#     def sna(data):
#         # List untuk menyimpan tweet yang mengandung tag '@' dan usernamenya
#         tagged_tweets = []

#         # Ekstraksi tweet yang mengandung tag '@' dan usernamenya
#         for row in data:
#             full_text = row['full_text']
#             username = row['username']
#             predicted_label = row['predicted_label']

#             # Gunakan ekspresi reguler untuk mencari tag '@' dan usernamenya
#             mentions = re.findall(r'@(\w+)', full_text)
#             # Jika ada tag '@', tambahkan ke list tagged_tweets
#             if mentions:
#                 for mention in mentions:
#                     tagged_tweets.append({
#                         'source': username,
#                         'target': mention,
#                         'tweet': full_text,
#                         'sentiment': predicted_label
#                     })

#         # Buat DataFrame dari list tagged_tweets
#         df = pd.DataFrame(tagged_tweets)

#         # List untuk menyimpan data dengan atribut weight
#         weighted_data = []

#         # Atribut weight dasar
#         base_weight = 1

#         # Ekstraksi data dan hitung weight
#         for index, row in df.iterrows():
#             source = row['source']
#             target = row['target']

#             # Cari apakah data dengan source dan target yang sama sudah ada di weighted_data
#             existing_data_index = None
#             for i, data in enumerate(weighted_data):
#                 if data['source'] == source and data['target'] == target:
#                     existing_data_index = i
#                     break

#             if existing_data_index is not None:
#                 # Jika data sudah ada, tambahkan 1 pada atribut weight
#                 weighted_data[existing_data_index]['weight'] += 1
#             else:
#                 # Jika data belum ada, tambahkan data baru dengan weight dasar
#                 weighted_data.append({'source': source, 'target': target, 'weight': base_weight})

#         # Buat DataFrame dari list weighted_data
#         weighted_df = pd.DataFrame(weighted_data)

#         # Buat graf dari DataFrame
#         G = nx.from_pandas_edgelist(weighted_df, 'source', 'target', ['weight'])

#         # Hitung betweenness centrality
#         bc_values = nx.betweenness_centrality(G)

#         # Hitung eigenvector centrality
#         ev_values = nx.eigenvector_centrality(G)

#         # Buat DataFrame dari hasil betweenness centrality
#         bc_df = pd.DataFrame(list(bc_values.items()), columns=['account_name', 'BC'])

#         # Urutkan DataFrame berdasarkan kolom 'BC' secara descending
#         bc_df = bc_df.sort_values(by='BC', ascending=False)

#         # Buat DataFrame dari hasil eigenvector centrality
#         ev_df = pd.DataFrame(list(ev_values.items()), columns=['account_name', 'EV'])

#         # Urutkan DataFrame berdasarkan kolom 'EV' secara descending
#         ev_df = ev_df.sort_values(by='EV', ascending=False)

#         # Normalisasi menggunakan Min-Max Scaling untuk kolom 'EV'
#         scaler_ev = MinMaxScaler(feature_range=(0, 1))
#         ev_df['norm_EV'] = ev_df['EV'].rank() / len(ev_df)

#         # Normalisasi menggunakan Min-Max Scaling untuk kolom 'BC'
#         scaler_bc = MinMaxScaler()
#         bc_df['norm_BC'] = bc_df['BC'].rank() / len(bc_df)

#         # Gabungkan DataFrame bc_df dan ev_df berdasarkan kolom 'account_name'
#         result_df = pd.merge(bc_df, ev_df, on='account_name', how='outer')

#         # Susun kolom sesuai dengan urutan yang diinginkan
#         result_df = result_df[['account_name', 'BC', 'EV', 'norm_BC', 'norm_EV']]

#         # Buat atribut baru 'final_measure' yang berisikan rata-rata dari 'norm_BC' dan 'norm_EV'
#         result_df['final_measure'] = (result_df['norm_BC'] + result_df['norm_EV']) / 2

#         # Urutkan DataFrame berdasarkan kolom 'final_measure' secara descending
#         result_df = result_df.sort_values(by='final_measure', ascending=False)

#         # Ambil top 10 nilai 'final_measure'
#         top_10_final_measure = result_df.head(10)

#         # Tampilkan tabel dengan urutan nomor dan account_name
#         top_10_final_measure_table = top_10_final_measure[['account_name']].reset_index(drop=True)
#         top_10_final_measure_table.index += 1

#         # Create a networkx graph
#         G = nx.from_pandas_edgelist(df, 'source', 'target')

#         # Ekstrak top 10 account names dari DataFrame
#         top_account_names = top_10_final_measure['account_name'].tolist()
#         print(top_account_names)
        
#         # Define the scale factor for node sizes
#         scale_factor_top = 30  # Adjust this value as needed
#         scale_factor_default = 15  # Adjust this value as needed for default nodes

#         # Define the color for the nodes based on sentiment
#         node_color_map = {'Negatif': '#FF6347', 'Positif': '#32CD32'}

#         # Create a pyvis network object
#         graph = Network(notebook=True)

#         # Add nodes and edges to the pyvis graph with specific size and color
#         for node in G.nodes:
#             # Ambil tweet yang terkait dengan node ini
#             tweets = df[df['source'] == node]['tweet'].tolist()
#             sentiments = df[df['source'] == node]['sentiment'].tolist()
#             print(f'Node: {node}, Sentiments: {sentiments}')  # Debugging line to check sentiments
#             if sentiments:
#                 color = node_color_map[sentiments[0]]  # Assume all sentiments are the same for simplicity
#             else:
#                 color = '#D3D3D3'  # Default color if no sentiment is found
#             size = scale_factor_top if node in top_account_names else scale_factor_default
#             graph.add_node(node, size=size, color=color, title='<br>'.join(tweets))

#         for edge in G.edges:
#             graph.add_edge(edge[0], edge[1])

#         # Set default options
#         options = {
#             "nodes": {
#                 "shape": 'dot',
#                 "font": {
#                     "size": 12
#                 }
#             }
#         }

#         # Convert options to a JSON-formatted string
#         options_str = str(options).replace("'", '"')

#         # Show the graph with options
#         graph.set_options(options_str)
#         graph.show('graph.html')

#         return top_10_final_measure_table


In [35]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
import re
from sklearn.preprocessing import MinMaxScaler
import community as community_louvain  # Import Louvain method for community detection

class Sna:
    @staticmethod
    def sna(data):
        # List untuk menyimpan tweet yang mengandung tag '@' dan usernamenya
        tagged_tweets = []

        # Ekstraksi tweet yang mengandung tag '@' dan usernamenya
        for _, row in data.iterrows():
            full_text = row['full_text']
            username = row['username']
            predicted_label = row['predicted_label']

            # Gunakan ekspresi reguler untuk mencari tag '@' dan usernamenya
            mentions = re.findall(r'@(\w+)', full_text)
            # Jika ada tag '@', tambahkan ke list tagged_tweets
            if mentions:
                for mention in mentions:
                    tagged_tweets.append({
                        'source': username,
                        'target': mention,
                        'tweet': full_text,
                        'sentiment': predicted_label
                    })

        # Buat DataFrame dari list tagged_tweets
        df = pd.DataFrame(tagged_tweets)

        # Buat graf dari DataFrame
        G = nx.from_pandas_edgelist(df, 'source', 'target')

        # Deteksi komunitas menggunakan algoritma Louvain
        partition = community_louvain.best_partition(G)
        
        # Assign community to each node
        community_dict = {node: community for node, community in partition.items()}

        # Ekstrak top 10 account names dari DataFrame berdasarkan jumlah koneksi
        degree_dict = dict(G.degree(G.nodes))
        top_account_names = sorted(degree_dict, key=degree_dict.get, reverse=True)[:10]

        # Define the scale factor for node sizes
        scale_factor_top = 30  # Adjust this value as needed
        scale_factor_default = 15  # Adjust this value as needed for default nodes

        # Define colors for communities
        community_colors = ['#1E90FF', '#FFD700', '#8A2BE2', '#FF7F50', '#7FFF00', '#FF6347', '#32CD32', '#D3D3D3']

        # Create a pyvis network object
        graph = Network(notebook=True, directed=True)

        # Add nodes and edges to the pyvis graph with specific size and color
        for node in G.nodes:
            # Ambil tweet yang terkait dengan node ini
            tweets = df[df['source'] == node]['tweet'].tolist()
            community_color = community_colors[community_dict[node] % len(community_colors)]
            size = scale_factor_top if node in top_account_names else scale_factor_default
            graph.add_node(node, size=size, color=community_color, title='<br>'.join(tweets))

        for edge in G.edges:
            graph.add_edge(edge[0], edge[1])

        # Set default options
        options = {
            "nodes": {
                "shape": 'dot',
                "font": {
                    "size": 12
                }
            }
        }

        # Convert options to a JSON-formatted string
        options_str = str(options).replace("'", '"')

        # Show the graph with options
        graph.set_options(options_str)
        graph.show('graph.html')

        return df

# Baca data dari CSV
data = pd.read_csv('./hasil-klasifikasi/klasifikasi_timnas_mainnya_new.csv')

# Buat visualisasi jaringan
sna = Sna()
sna.sna(data)


graph.html


,source,target,tweet,sentiment
0,erseee_,safergieway,@safergieway @PSSI True bang. bener² ga ngerti...,Negatif
1,erseee_,PSSI,@safergieway @PSSI True bang. bener² ga ngerti...,Negatif
2,IjoelSi,garagarabola_,@garagarabola_ Lebih penting timnas bagus main...,Positif
3,iraone2345,Tyows,@Tyows pdhal Pemain Timnas inggris nih gw rasa...,Negatif
4,Dwifitr70287374,utdfocusid,@utdfocusid @bliblidotcom Semuanya out kecuali...,Negatif
...,...,...,...,...
733,pongky1971,bliblidotcom,@idextratime @bliblidotcom Mantap mainnya timn...,Positif
734,IlsyahSendra,FaktaSepakbola,@FaktaSepakbola Menang sh tp apa daya.. Sumpah...,Positif
735,ultinterpersiba,SiaranBolaLive,@SiaranBolaLive Apresiasi utk ketahanan fisik ...,Positif
736,ultinterpersiba,Indostransfer,@Indostransfer Apresiasi utk ketahanan fisik t...,Positif


In [33]:
import re

# Assuming data is loaded into a DataFrame `data`
data = pd.read_csv('./hasil-klasifikasi/klasifikasi_timnas_mainnya_new.csv')

# Prepare data for SNA
dataForSna = []
for index, tweet in data.iterrows():
    dataForSna.append({
        "username": tweet['username'],
        "full_text": tweet['full_text'],
        "predicted_label": tweet['predicted_label']
    })

buzzer = Sna.sna(dataForSna)['account_name'].values.tolist()

# Membuat dictionary untuk menyimpan tweet berdasarkan nama pengguna
tweets_by_user = {}

# Mengelompokkan tweet berdasarkan nama pengguna
for tweet_data in dataForSna:
    username = tweet_data['username']
    full_text = tweet_data['full_text']

    if username in tweets_by_user:
        tweets_by_user[username].append(full_text)
    else:
        tweets_by_user[username] = [full_text]

# Menampilkan tweet yang berhubungan dengan setiap buzzer
for buzzer_username in buzzer:
    print(f"Tweet yang berhubungan dengan {buzzer_username}:")
    if buzzer_username in tweets_by_user:
        for tweet in tweets_by_user[buzzer_username]:
            print(tweet)
    else:
        print("Tidak ada tweet yang berhubungan.")

# Iterasi untuk setiap buzzer
for buz in buzzer:
    print(f"Tweet yang berisi tag untuk akun {buz}:")
    for tweet_data in dataForSna:
        full_text = tweet_data['full_text']
        mentions = re.findall(r'@(\w+)', full_text)
        if buz in mentions or tweet_data['username'] == buz:
            print(full_text)
    print("\n")


['idextratime', 'bliblidotcom', 'moechaaliem', 'skyrim8764801', 'txtfromkirana', 'biantoro_robin', 'frans_natadi', 'lamina1122', 'UekyAgung', 'nareshvaraaa']
graph.html
Tweet yang berhubungan dengan idextratime:
Half-time: Arab Saudi 0-0 Korea Selatan  Lebih meyakinkan mainnya timnas Indonesia ga sih  #Blibli @bliblidotcom
Tweet yang berhubungan dengan bliblidotcom:
Tidak ada tweet yang berhubungan.
Tweet yang berhubungan dengan moechaaliem:
@ghozali0312 @indosupporter IS dulu pernah punya tim yg keren diantara semua kelompok umur timnas yakni u19 eranya Evan Dimas, smpe saat ini bhkn belum nemu lagi yg kyak gitu Mainnya pun bnyak bola pendek, squad pun dibentuk dri keliling nyari pemain, knpa skrng ngoyo ya? Apa mungkin bnyakan pemain titipan"
@NurHasa52632749 @erirendy5 @Indostransfer Tpi di era itu timnas kita msih menangan klo ketemu mereka, beda jauh ketika dilatih si engkong, hasil paling bagus ya seri, itupun di kandang kita mainnya
@cco_archive @gberrei @Indostransfer Ngapain n

In [ ]:
import tweepy
import re
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from textblob import TextBlob

data = pd.read_csv('./hasil-klasifikasi/klasifikasi_timnas_mainnya_new.csv')

# Extract data
data = []
for tweet in tweets:
    data.append({'username': tweet.user.screen_name, 'full_text': tweet.full_text})

# Create DataFrame
df = pd.DataFrame(data)

# Sentiment analysis
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'Positif'
    elif analysis.sentiment.polarity == 0:
        return 'Netral'
    else:
        return 'Negatif'

df['sentiment'] = df['full_text'].apply(get_sentiment)

# Extract mentions
tagged_tweets = []
for index, row in df.iterrows():
    mentions = re.findall(r'@(\w+)', row['full_text'])
    for mention in mentions:
        tagged_tweets.append({'source': row['username'], 'target': mention, 'tweet': row['full_text'], 'sentiment': row['sentiment']})

tagged_df = pd.DataFrame(tagged_tweets)

# Build graph
G = nx.from_pandas_edgelist(tagged_df, 'source', 'target', ['sentiment'])

# Community detection
communities = nx.algorithms.community.label_propagation_communities(G)
community_dict = {node: i for i, community in enumerate(communities) for node in community}

# Draw graph
pos = nx.spring_layout(G, k=0.15, iterations=20)
plt.figure(figsize=(12, 12))

colors = {'Positif': '#32CD32', 'Negatif': '#FF6347'}
node_colors = [colors[tagged_df[tagged_df['source'] == node]['sentiment'].values[0]] if node in tagged_df['source'].values else '#D3D3D3' for node in G.nodes]

nx.draw_networkx_nodes(G, pos, node_size=50, node_color=node_colors, alpha=0.7)
nx.draw_networkx_edges(G, pos, alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=8)

plt.title('Twitter Network: Prabowo-Gibran')
plt.show()


## Community

In [14]:
# import networkx as nx
# import pandas as pd
# import re
# from textblob import TextBlob
# from collections import defaultdict
# from pyvis.network import Network
# from sklearn.preprocessing import MinMaxScaler

# class Sna:
#     def sna(data):    
#         # List untuk menyimpan tweet yang mengandung tag '@' dan usernamenya
#         tagged_tweets = []

#         # Ekstraksi tweet yang mengandung tag '@' dan usernamenya
#         for row in data:
#             full_text = row['full_text']
#             username = row['username']

#             # Gunakan ekspresi reguler untuk mencari tag '@' dan usernamenya
#             mentions = re.findall(r'@(\w+)', full_text)
#             # Jika ada tag '@', tambahkan ke list tagged_tweets
#             if mentions:
#                 for mention in mentions:
#                     tagged_tweets.append({'source': username, 'target': mention, 'tweet': full_text})

#         # Buat DataFrame dari list tagged_tweets
#         df = pd.DataFrame(tagged_tweets)

#         # List untuk menyimpan data dengan atribut weight
#         weighted_data = []

#         # Atribut weight dasar
#         base_weight = 1

#         # Ekstraksi data dan hitung weight
#         for index, row in df.iterrows():
#             source = row['source']
#             target = row['target']

#             # Cari apakah data dengan source dan target yang sama sudah ada di weighted_data
#             existing_data_index = None
#             for i, data in enumerate(weighted_data):
#                 if data['source'] == source and data['target'] == target:
#                     existing_data_index = i
#                     break

#             if existing_data_index is not None:
#                 # Jika data sudah ada, tambahkan 1 pada atribut weight
#                 weighted_data[existing_data_index]['weight'] += 1
#             else:
#                 # Jika data belum ada, tambahkan data baru dengan weight dasar
#                 weighted_data.append({'source': source, 'target': target, 'weight': base_weight})

#         # Buat DataFrame dari list weighted_data
#         weighted_df = pd.DataFrame(weighted_data)

#         # Buat graf dari DataFrame
#         G = nx.from_pandas_edgelist(weighted_df, 'source', 'target', ['weight'])

#         # Deteksi Komunitas menggunakan algoritma Louvain
#         import community as community_louvain
#         partition = community_louvain.best_partition(G)

#         # Tambahkan partisi (komunitas) ke DataFrame
#         df['community'] = df['source'].map(partition)

#         # Analisis Sentimen untuk setiap pengguna
#         user_sentiments = {}
#         for tweet in data:
#             user = tweet['username']
#             sentiment = TextBlob(tweet['full_text']).sentiment.polarity
#             user_sentiments[user] = 'positive' if sentiment > 0 else 'negative' if sentiment < 0 else 'neutral'

#         # Pemetaan Sentimen ke Komunitas
#         community_sentiments = defaultdict(list)
#         for idx, community in partition.items():
#             if idx in user_sentiments:
#                 community_sentiments[community].append(user_sentiments[idx])

#         # Mengukur Homogenitas Sentimen
#         for community, sentiments in community_sentiments.items():
#             positive_count = sentiments.count("positive")
#             negative_count = sentiments.count("negative")
#             total = len(sentiments)
#             homogeneity = max(positive_count, negative_count) / total if total > 0 else 0
#             print(f"Community {community} has a homogeneity of {homogeneity:.2f}")

#         # Visualisasi Graf Jaringan dengan Pyvis
#         node_color_map = {0: '#ff9999', 1: '#66b3ff', 2: '#99ff99', 3: '#ffcc99'}
#         graph = Network(notebook=True)

#         for node in G.nodes:
#             community = partition[node]
#             color = node_color_map.get(community, '#b3b3b3')
#             graph.add_node(node, color=color, title=node)

#         for edge in G.edges:
#             graph.add_edge(edge[0], edge[1])

#         # Set default options
#         options = {
#             "nodes": {
#                 "shape": 'dot',
#                 "font": {
#                     "size": 12
#                 }
#             }
#         }

#         # Convert options to a JSON-formatted string
#         options_str = str(options).replace("'", '"')

#         # Show the graph with options
#         graph.set_options(options_str)
#         graph.show('graph.html')

#         return community_sentiments

# # Data example
# dataForSna = []
# for index, tweet in data.iterrows():
#     dataForSna.append({
#         "username": tweet['username'],
#         "full_text": tweet['full_text']
#     })

# buzzer = Sna.sna(dataForSna)

# # Print sentiment analysis results
# for community, sentiments in buzzer.items():
#     print(f"Community {community} sentiments: {sentiments}")


TypeError: string indices must be integers